<a href="https://colab.research.google.com/github/Coyote-Schmoyote/movie-summary-generator-ja/blob/main/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
from tensorflow import keras

In [15]:
data = pd.read_csv("/content/drive/MyDrive/E資格機械学習プロジェクト/文章生成/movie-synopsis-ja.csv")

In [3]:
data.head()

,タイトル,あらすじ
0,ショーシャンクの空に,1947年、メイン州ポートランド。若くして銀行副頭取を務める優秀な銀行員アンドリュー・デュフ...
1,ゴッドファーザー,第二次世界大戦終戦直後の1945年。ニューヨーク五大ファミリーの一角で、最大の勢力を誇るイタ...
2,ダークナイト,素性不明の犯罪者ジョーカーに雇われた、道化師のマスクを被った男たちがゴッサム・シティの銀行を...
3,ロード・オブ・ザ・リング/王の帰還,昼なお暗いモルドールの山。フロドが眠る隣でゴラムは、自身が“愛しいしと”と呼ぶ“一つの指輪”...
4,シンドラーのリスト,1939年9月、ドイツ軍によりポーランドが占領され、ポーランドの都市クラクフもドイツ軍の占領...


In [4]:
data["あらすじ"][0]

'1947年、メイン州ポートランド。若くして銀行副頭取を務める優秀な銀行員アンドリュー・デュフレーン （アンディ）は、妻とその愛人を射殺した罪に問われる。無実を訴えるも終身刑の判決が下り、劣悪なショーシャンク刑務所への服役が決まる。ショーシャンクでは、長年服役する「調達屋」ことエリス・ボイド・レディング（レッド）が、もう何度目かとなる仮釈放の審査を受け、更生したことを訴えるがやはり却下される。レッドが落胆し部屋を出ると、アンディを含む新しい受刑者達が護送されて来る。アンディら新入り達はノートン所長とハドリー主任刑務官から脅しを含めたショーシャンク刑務所の紹介をされ、その晩に取り乱した一人の新人受刑者がハドリーから過剰暴力を受けて死んでしまう。\n孤立していたアンディはやがてレッドに声をかけ、鉱物採集の趣味のため小さなロックハンマーを注文する。それをきっかけにアンディはレッドと交友を重ね始める。他方、アンディは荒くれ者のボグズとその一味に性行為を強要され、抵抗のため常に生傷が絶えない生活が続いた。\n1949年、アンディは屋根の修理作業中、ハドリーの遺産相続問題を知り、財務経理の知識を駆使し作業仲間達へのビールと引き換えに解決策を提案する。ビールを手に入れ仲間達から尊敬される一方で、ハドリーら刑務官からも一目置かれるようになる。その後ボグズらがアンディを襲って全治1ヶ月の重傷を負わせるも、彼はハドリーに半殺しにされ、以後、アンディを襲う者はいなくなる。アンディが治療を終え自分の房に戻ってくると、レッドに注文していたリタ・ヘイワースの大判ポスターが退院祝いとして置かれていた。\nやがて、アンディは図書係に配置換えとなり、もう50年も服役している老囚人ブルックスの助手となる。だが、その本当の目的は所長や刑務官達の税務処理や資産運用をアンディに行わせるためだった。アンディは有能な銀行家としての手腕を発揮する一方で、名ばかりだった図書係としても精力的に活動を始め、州議会に図書館予算の請求を毎週送るようになる。\n1954年、ブルックスに仮釈放の許可が下りるが、50年服役した老人は塀の外の生活への恐れから取り乱す。アンディらに説得され、仮釈放を受け入れるが、結局、外の生活に馴染むことはできず、最期は首を吊って死んでしまう。死の間際に送られた感謝の手紙を読んでアンディとレッドは苛

In [16]:
data.columns = ["Title", "Summary"]

In [35]:
data.columns

Index(['Title', 'Summary'], dtype='object')

In [36]:
data.isnull().sum()

Title      0
Summary    1
dtype: int64

In [17]:
data = data.dropna(axis=0)

In [38]:
data.isnull().sum()

Title      0
Summary    0
dtype: int64

In [39]:
data.shape

(99, 2)

In [40]:
data.dtypes

Title      object
Summary    object
dtype: object

## Data Processing

In [12]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
#dictionary
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!pip install spacy[ja]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install -U ginza ja-ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.3 MB/s 
  Attempting uninstall: ginza
    Found existing installation: ginza 5.1.1
    Uninstalling ginza-5.1.1:
      Successfully uninstalled ginza-5.1.1


In [24]:
import MeCab
import spacy
from spacy.lang.ja import Japanese
from spacy import displacy
import neologdn
import pkg_resources, imp

In [6]:
nlp = spacy.load('ja_ginza')

In [7]:
wakati = MeCab.Tagger("-Owakati")

In [25]:
title = wakati.parse(data["Title"][0]).split()

In [26]:
title

['ショーシャンク', 'の', '空', 'に']

In [27]:
title = " ".join(title)

In [28]:
title

'ショーシャンク の 空 に'

In [29]:
title = neologdn.normalize(title)

In [30]:
title

'ショーシャンクの空に'

In [33]:
nlp = Japanese()

doc = nlp(title)
for token in doc:
  print(token.text)


ショーシャンク
の
空
に


In [34]:
print([t.text for t in doc])
print([t.lemma_ for t in doc])
print([t.pos_ for t in doc])

['ショーシャンク', 'の', '空', 'に']
['ショーシャンク', 'の', '空', 'に']
['PROPN', 'ADP', 'NOUN', 'ADP']


In [41]:
def ja_preprocess(text):
    nlp = Japanese()
    text = text[:500]
    text = BeautifulSoup(text).get_text()
    text = text.lower()
    text = neologdn.normalize(text)
    text = nlp(text)
    return text

In [43]:
summary = data["Summary"].apply(lambda x: ja_preprocess(x))

In [51]:
summary[0]

1947年、メイン州ポートランド。若くして銀行副頭取を務める優秀な銀行員アンドリュー・デュフレーン(アンディ)は、妻とその愛人を射殺した罪に問われる。無実を訴えるも終身刑の判決が下り、劣悪なショーシャンク刑務所への服役が決まる。ショーシャンクでは、長年服役する「調達屋」ことエリス・ボイド・レディング(レッド)が、もう何度目かとなる仮釈放の審査を受け、更生したことを訴えるがやはり却下される。レッドが落胆し部屋を出ると、アンディを含む新しい受刑者達が護送されて来る。アンディら新入り達はノートン所長とハドリー主任刑務官から脅しを含めたショーシャンク刑務所の紹介をされ、その晩に取り乱した一人の新人受刑者がハドリーから過剰暴力を受けて死んでしまう。
孤立していたアンディはやがてレッドに声をかけ、鉱物採集の趣味のため小さなロックハンマーを注文する。それをきっかけにアンディはレッドと交友を重ね始める。他方、アンディは荒くれ者のボグズとその一味に性行為を強要され、抵抗のため常に生傷が絶えない生活が続いた。
1949年、アンディは屋根の修理作業中、ハドリーの遺産相続問題を知り、財務経理の知識を駆使し作

In [49]:
print([token.text for token in summary[0][:10]])
print([token.pos_ for token in summary[0][:10]])

['1947', '年', '、', 'メイン', '州', 'ポートランド', '。', '若く', 'し', 'て']
['NUM', 'NOUN', 'PUNCT', 'PROPN', 'NOUN', 'PROPN', 'PUNCT', 'ADJ', 'VERB', 'SCONJ']


In [85]:
summary

0     (1947, 年, 、, メイン, 州, ポートランド, 。, 若く, し, て, 銀行, ...
1     (第, 二, 次, 世界, 大戦, 終戦, 直後, の, 1945, 年, 。, ニューヨー...
2     (素性, 不明, の, 犯罪, 者, ジョーカー, に, 雇わ, れ, た, 、, 道化, ...
3     (昼, なお, 暗い, モルドール, の, 山, 。, フロド, が, 眠る, 隣, で, ...
4     (1939, 年, 9, 月, 、, ドイツ, 軍, に, より, ポーランド, が, 占領...
                            ...                        
94    (広告, 会社, の, 重役, ロジャー, ・, ソーンヒル, は, 、, ホテル, の, ...
95    (「, スコティ, 」, こと, ジョン, ・, ファーガソン, 刑事, は, 、, 犯人,...
96    (サイレント, 映画, 全盛, の, 時代, 、, 俳優, ドン, と, 大, 女優, リナ...
97    (暗く, 荒廃, し, た, 大, 邸宅, 「, ザナドゥ, (, 英語, 版, ), 城,...
98    (1930, 年, 代, の, ベルリン, で, 、, 幼い, 少女, ばかり, を, 狙っ...
Name: Summary, Length: 99, dtype: object

In [86]:
chars = sorted(list(set(summary[0])))

In [87]:
len(chars)

283

In [108]:
flattened = []
for sublist in summary:
  for token in sublist:
    flattened.append(token)
print(len(flattened))

27827


In [109]:
vocabulary = []

for item in flattened:
  if not item in vocabulary:
    vocabulary.append(item)
print(len(vocabulary))

27827


In [110]:
char_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_char = dict((i, c) for i, c in enumerate(vocabulary))

In [113]:
maxlen = 8

step = 1

sentences = []

next_chars = []

for i in range(0, len(flattened) - maxlen, step):
    sentences.append(flattened[i: i + maxlen])
    next_chars.append(flattened[i + maxlen])
    
print(len(sentences))

27819


In [ ]:
#bag of words?